In [ ]:
%matplotlib inline
from parcels import FieldSet, Field, VectorField, ParticleFile, ParticleSet, JITParticle, AdvectionRK4, AdvectionEE, plotTrajectoriesFile, Variable, BrownianMotion2D
from parcels import ErrorCode
from parcels import rng as random
import numpy as np
import math
from datetime import timedelta, datetime
from operator import attrgetter

In [ ]:
# Load in ocean model  Copernicus
filenames = {'U': "Copernicus/stokes_drift/global-analysis-forecast-phy-001-024-hourly-merged-uv_1605607248932.nc",
             'V': "Copernicus/stokes_drift/global-analysis-forecast-phy-001-024-hourly-merged-uv_1605607248932.nc"}
variables = {'U': 'vsdx',
             'V': 'vsdy'}
dimensions = {'lat': 'latitude',
              'lon': 'longitude',
              'time': 'time'}

fieldset = FieldSet.from_netcdf(filenames, variables, dimensions, allow_time_extrapolation=True)

In [ ]:
class SampleParticle(JITParticle):  # Define a new particle class that contains three extra variables
    age = Variable('age', dtype=np.float32, initial=0.) # initialise age
    distance = Variable('distance', initial=0., dtype=np.float32)  # the distance travelled
    prev_lon = Variable('prev_lon', dtype=np.float32, to_write=False,
                        initial=attrgetter('lon'))  # the previous longitude
    prev_lat = Variable('prev_lat', dtype=np.float32, to_write=False,
                        initial=attrgetter('lat'))  # the previous latitude.

def SampleDistance(particle, fieldset, time): # Function measuring distance
    # Calculate the distance in latitudinal direction (using 1.11e2 kilometer per degree latitude)
    lat_dist = (particle.lat - particle.prev_lat) * 1.11e2
    # Calculate the distance in longitudinal direction, using cosine(latitude) - spherical earth
    lon_dist = (particle.lon - particle.prev_lon) * 1.11e2 * math.cos(particle.lat * math.pi / 180)
    # Calculate the total Euclidean distance travelled by the particle
    particle.distance += math.sqrt(math.pow(lon_dist, 2) + math.pow(lat_dist, 2))

    particle.prev_lon = particle.lon  # Set the stored values for next iteration.
    particle.prev_lat = particle.lat
        
def DeleteParticle(particle, fieldset, time): # Function that deletes particle if it goes out of bounds to avoid OutOfBounds error
    particle.delete()

In [ ]:
pset = ParticleSet.from_list(fieldset=fieldset,   # the fields on which the particles are advected
                             pclass=SampleParticle,  # the type of particles (JITParticle or ScipyParticle)
                             lon=[18.3176], 
                             lat=[-34.1395],
                             depth = None,
                             lonlatdepth_dtype= np.float32,
                             repeatdt=None)  

In [ ]:
# Minimum Hydrodynamic drag kernel 
def WaterDrag(particle, fieldset, time):
        (uvdx, uvdy) = fieldset.UV[time, particle.depth, particle.lat, particle.lon]
        Ac = 122.97 # cross-sectional area of plant, must be entered manually and calculated before hand
        mass = 17.10 # total mass of the kelp plant being simulated
        expo = 1
        waterdrag = ((0.5*1.025*(uvdx*uvdx)*0.47*Ac)*expo)/mass # syntax for calculating the loss in velocity as a result of drag
        particle.lon += (uvdx - waterdrag) # overall velocity minus the loss in the velocity as a result of drag
        particle.lat += (uvdx - waterdrag) # overall velocity minus the loss in the velocity as a result of drag

In [ ]:
# Mean Hydrodynamic drag kernel
def WaterDrag(particle, fieldset, time):
        (uvdx, uvdy) = fieldset.UV[time, particle.depth, particle.lat, particle.lon]
        Ac = 228.61 # cross-sectional area of plant, must be entered manually and calculated before hand
        mass = 34.5 # total mass of the kelp plant being simulated
        expo = 1
        waterdrag = ((0.5*1.025*(uvdx*uvdx)*0.47*Ac)*expo)/mass # syntax for calculating the loss in velocity as a result of drag
        particle.lon += (uvdx - waterdrag) # overall velocity minus the loss in the velocity as a result of drag
        particle.lat += (uvdy - waterdrag) # overall velocity minus the loss in the velocity as a result of drag

In [ ]:
# Maximum Hydrodynamic drag kernel
def WaterDrag(particle, fieldset, time):
        (uvdx, uvdy) = fieldset.UV[time, particle.depth, particle.lat, particle.lon]
        Ac = 386.28 # cross-sectional area of plant, must be entered manually and calculated before hand
        mass = 48.65 # total mass of the kelp plant being simulated
        expo = 1
        waterdrag = ((0.5*1.025*(uvdx*uvdx)*0.47*Ac)*expo)/mass # syntax for calculating the loss in velocity as a result of drag
        particle.lon += (uvdx - waterdrag) # overall velocity minus the loss in the velocity as a result of drag
        particle.lat += (uvdy - waterdrag) # overall velocity minus the loss in the velocity as a result of drag

In [ ]:
## define output file, select outfile according to simulation type

#output_file = pset.ParticleFile(name="RK4_PFloat_stokesdrift.nc", outputdt=timedelta(hours=1)) #check depth setting

output_file = pset.ParticleFile(name="Min_KFloat_stokesdrift.nc", outputdt=timedelta(hours=1)) #check depth setting
#output_file = pset.ParticleFile(name="Mean_KFloat_stokesdrift.nc", outputdt=timedelta(hours=1)) #check depth setting
#output_file = pset.ParticleFile(name="Max_KFloat_stokesdrift.nc", outputdt=timedelta(hours=1)) #check depth setting


## cast kernels, select appropriate 'kernel set'

#kernels = pset.Kernel(AdvectionRK4) + SampleDistance # for passive RK4 simulation
kernels = pset.Kernel(AdvectionRK4) + SampleDistance + WaterDrag # KelpFloat

## execute simulation
pset.execute(kernels, 
             runtime=timedelta(days=10),
             dt=timedelta(hours=1),
             output_file=output_file,
             recovery = {ErrorCode.ErrorOutOfBounds: DeleteParticle})

In [ ]:
plotTrajectoriesFile('RK4_PFloat_stokesdrift.nc');
plotTrajectoriesFile('Min_KFloat_stokesdrift.nc');
plotTrajectoriesFile('Mean_KFloat_stokesdrift.nc');
plotTrajectoriesFile('Max_KFloat_stokesdrift.nc');

In [ ]:
# animation

#plotTrajectoriesFile('RK4_PFloat_stokesdrift.nc', mode='movie2d_notebook')

#plotTrajectoriesFile('Min_KFloat_stokesdrift.nc', mode='movie2d_notebook')
#plotTrajectoriesFile('Mean_KFloat_stokesdrift.nc', mode='movie2d_notebook')
#plotTrajectoriesFile('Max_KFloat_stokesdrift.nc', mode='movie2d_notebook')
